In [1]:
import os
import random

import cv2
import numpy as np

In [2]:
class utils:
    def __init__(self):
        pass

    def load_dataset(dataset_path):
        folder_list = sorted(os.listdir(dataset_path))
        folder_path = []
        class_label = np.array([], dtype=np.int16)
        class_dictionary = {}
        for i, folder_name in enumerate(folder_list):
            class_folder_path = os.path.join(dataset_path, folder_name)
            list_image_name = sorted(os.listdir(class_folder_path))
            temp_folder_path = [os.path.join(class_folder_path, image_name) for image_name in list_image_name]
            
            folder_path += temp_folder_path
            temp_class_label = np.full(len(list_image_name), i, dtype=np.int16)
            class_label = np.concatenate((class_label, temp_class_label), axis=0)
            class_dictionary[str(i)] = folder_name

        return np.asarray(folder_path), class_label, class_dictionary

    def convert_image_to_matrix(folder_path):
        list_of_image_matrix = []
        size= (500, 500) #ukuran diubah jadi 1 x 1 dengan ukuran matrik 500 x 500

        for file_img in folder_path:
            image = cv2.imread(file_img, 1)
            image_matrix = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_matrix = cv2.resize(image_matrix, size)
            list_of_image_matrix.append(image_matrix)
        # return array diubah dari (batch_size, height, width, channels) menjadi (batch_size, channels, height, width)
        # hal ini dilakukan untuk mempermudah memprosesan yang akan dilakukan
        return np.array(list_of_image_matrix).transpose(0, 3, 1, 2)

# untuk testing
# if __name__ == "__main__":
#     folder_path, class_label, class_dictionary = utils.load_dataset("./dataset")
#     print(utils.convert_image_to_matrix(folder_path))

In [3]:
class Model:
    def __init__(
        self,
        input_size: list[list[int]],
        padding_size: int,
        weights: list[list[list[float]]],
        conv_filter_count: int,
        conv_filter_size: list[tuple[int]],
        conv_stride_size: list[int],
        pool_filter_size: tuple[int],
        pool_stride_size: int,
        pool_mode: str,
        dense_unit_count: int,
        dense_activation: str,
    ) -> None:
        self._input_size = input_size
        self._padding_size = padding_size
        self._weights = weights
        self._conv_filter_count = conv_filter_count
        self._conv_filter_size = conv_filter_size
        self._conv_stride_size = conv_stride_size
        self._pool_filter_size = pool_filter_size
        self._pool_stride_size = pool_stride_size
        self._pool_mode = pool_mode
        self._dense_unit_count = dense_unit_count
        self._dense_activation = dense_activation
        self._convolution_layers = [
            self.ConvolutionLayer(self._conv_filter_size)
            for _ in range(self._conv_filter_count)
        ]
        self._detector_layer = self.DetectorLayer()
        self._pooling_layer = self.PoolingLayer()
        self._dense_layer = self.DenseLayer()
        self._flatten_layer = self.FlattenLayer()

    class ConvolutionLayer:
        def __init__(self, filter_size: list[tuple[int]]) -> None:
            self._filter_weight = [
                [
                    [random.random() for _ in range(len(filter_size[i][j]))]
                    for j in range(len(filter_size[i]))
                ]
                for i in range(len(filter_size))
            ]

        def convolute(
            self, weights: list[list[list[float]]]
        ) -> list[list[list[float]]]:
            feature_maps = []
            for i in range(len(weights)):
                feature_map = []
                for j in range(len(weights[i]) - len(self._filter_weight[i]) + 1):
                    feature_row = []
                    for k in range(
                        len(weights[i][j]) - len(self._filter_weight[i][j]) + 1
                    ):
                        field = np.array(
                            weights[i][j : j + len(self._filter_weight[i])][
                                k : k + len(self._filter_weight[i][j])
                            ]
                        )
                        feature = field * np.array(self._filter_weight[i])
                        feature_row.append(np.sum(feature))
                    feature_map.append(feature_row)
                feature_maps.append(feature_map)
            return feature_maps

    class DetectorLayer:
        def __init__(self) -> None:
            pass

        def detect(self, input) -> None:
            #Detector menggunakan fungsi relu
            return np.maximum(input, 0)

    class PoolingLayer:
        def __init__(self, filter_size, stride_size, mode) -> None:
            self.filter_size = filter_size
            self.stride_size = stride_size
            self.mode = mode

        def pool(self, input_matrix) -> None:
            depth, height, width = input_matrix.shape
            filter_height = (height - self.filter_size) // self.stride_size + 1
            filter_width = (width - self.filter_size) // self.stride_size + 1
            pooled = np.zeros([depth, filter_height, filter_width], dtype=np.double)
            for d in range(0, depth):
                for h in range(0, filter_height):
                    for w in range(0, filter_width):
                        if(self.mode == "average"):
                            pooled[d,h,w] = self.average(input_matrix, d, h, w)
                        elif(self.mode == "max"):
                            pooled[d,h,w] = self.max(input_matrix, d, h, w)
            return pooled

        def average(self, input_matrix, d, h, w):
            h_start = h * self.stride_size
            w_start = w * self.stride_size
            h_end = h_start + self.filter_size
            w_end = w_start + self.filter_size
            return np.average(input_matrix[d, h_start:h_end, w_start:w_end])

        def max(self, input_matrix, d, h, w):
            h_start = h * self.stride_size
            w_start = w * self.stride_size
            h_end = h_start + self.filter_size
            w_end = w_start + self.filter_size
            return np.max(input_matrix[d, h_start:h_end, w_start:w_end])
            
    class DenseLayer:
        def __init__(self, units, activation) -> None:
            self.units = units
            self.activation = activation
            self.bias = np.zeros(units)
            self.weight = np.random.randn(units)

        def dense(self, input_matrix) -> None:
            result = np.zeros(self.units)
            for i in range(self.units):
                input_weight = np.sum(self.weight[i] * input_matrix)
                result[i] = input_weight + self.bias[i]
            
            if(self.activation == "sigmoid"):
                return 1 / (1 + np.exp(-result))
            elif(self.activation == "relu"):
                return np.maximum(result, 0)

    class FlattenLayer:
        def __init__(self) -> None:
            pass

        def flatten(self, input_matrix) -> None:
            #dengan menggunakan flatten dari library numpy
            return input_matrix.flatten()

    def _pad_weights(self):
        for i in range(len(self._weights)):
            new_weight = [
                [
                    0.0 if j == 0 or k == 0 else self._weights[j - 1][k - 1]
                    for k in range(len(self._weights[i][j]))
                ]
                for j in range(len(self._weights[i]))
            ]
            self._weights = new_weight

    def feedforward(self) -> None:
        self._pad_weights()
        self._convolution_layers.convolute()
        self._detector_layer.detect()
        self._pooling_layer.pool()
        self._dense_layer.dense()
        self._flatten_layer.flatten()

    def back_propagate(self) -> None:
        pass

In [4]:
# model = Model()
# poolingLayer = model.PoolingLayer(2,2, "max")
# input_tensor = np.array([[[1, 2, 3, 4],
#                           [5, 6, 7, 8],
#                           [9, 10, 11, 12],
#                           [13, 14, 15, 16]],
#                          
#                          [[17, 18, 19, 20],
#                           [21, 22, 23, 24],
#                           [25, 26, 27, 28],
#                           [29, 30, 31, 32]],
#                          
#                          [[33, 34, 35, 36],
#                           [37, 38, 39, 40],
#                           [41, 42, 43, 44],
#                           [45, 46, 47, 48]]])
# 
# result = poolingLayer.pool(input_tensor)
# print("pooling layer :")
# print(result)
# dense = model.DenseLayer(4, "relu")
# result = dense.dense(result)
# print("dense layer :")
# print(result)
# flaten = model.FlattenLayer()
# print("flatten layer :")
# print(flaten.flatten(result))